In [1]:
import os
import re
from collections import defaultdict

In [3]:
'' == ''

True

In [4]:
import pandas as pd

data = defaultdict(lambda: {'topic': '', 'content': '', 'messages': [], 'month': ''})

for file_path in os.listdir("cleaned_intercom_messages"):
    csv = pd.read_csv(f"cleaned_intercom_messages/{file_path}")
    csv = csv[csv['clean_body'].notna()]
    bodies = csv["clean_body"]
    ids = zip(csv["key_intercomconversation"], csv["key_iteration"])
    for id, body in zip(ids, bodies):
        m = re.search(" Selected category: (.+) Content: (.+) uid:", body)
        if m is not None:
            data[id]['topic'] = m.group(1)
            data[id]['content'] = m.group(2)
        else:
            data[id]['messages'].append(body)
            if data[id]['month'] == '':
                data[id]['month'] = file_path.split("-")[1]

df = pd.DataFrame.from_dict(data, orient="index")
print(len(df[df["topic"] == '']))
print(len(df[df["topic"] != '']))

329564
10


In [63]:
df.head()

,topic,content,messages,month
13672140114_5,,,"[{{name}} im also talking ot this guy, lll, je...",01
13672140114_6,,,[Hm],01
13672140114_8,,,[🔗 Riley Young linked SE-{{number}} Blank emai...,04
13675529033_1,,,"[Guys, please help the customer when he comes ...",03
13675881994_3,,,[Record a video],10


In [17]:
"_".join(("1", "2"))

'1_2'

In [20]:
new_ids = ["_".join([str(tup[0]), str(tup[1])]) for tup in df.index]

In [31]:
df.index = new_ids

In [32]:
df.head()

,topic,content,messages,month
13672140114_5,,,"[{{name}} im also talking ot this guy, lll, je...",01
13672140114_6,,,[Hm],01
13672140114_8,,,[🔗 Riley Young linked SE-{{number}} Blank emai...,04
13675529033_1,,,"[Guys, please help the customer when he comes ...",03
13675881994_3,,,[Record a video],10


In [41]:
def get_monthly_data(original_data: pd.DataFrame, preprocessed_data: dict, month: str) -> dict:
    ids = original_data[original_data['month'] == month].index
    monthly_data = {}
    for _id in ids:
        if _id in preprocessed_data.keys():
            monthly_data[_id] = preprocessed_data[_id]
        
    return monthly_data

In [46]:
import json

In [52]:
with open("new_lems.json") as f:
    preprocessed = json.loads(f.read())

In [61]:
months = []

for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']:
    months.append(get_monthly_data(df, preprocessed, month))